In [1]:
!pip install --upgrade --no-cache-dir transformers==4.45.2 indic-transliteration -q
!pip install scikit-learn pandas tqdm matplotlib -q

import os, torch, random, numpy as np, json
from pathlib import Path
from tqdm.auto import tqdm

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

WORK_DIR = Path("/kaggle/working")
WORK_DIR.mkdir(exist_ok=True, parents=True)
print("✅ Device:", DEVICE)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 100.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.6/159.6 kB 296.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 136.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 290.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.1.1 requires pyarrow>=21.0.0, but you have pyarrow 19.0.1 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.
✅ Device: cuda


# Imports, Config

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

DATA_DIR = Path("/kaggle/input/dataaaaaa")
POS_FILE = DATA_DIR / "all_positive_8500.txt"
NEG_FILE = DATA_DIR / "all_negative_3307.txt"
assert POS_FILE.exists() and NEG_FILE.exists(), "Data files missing!"

def read_txt(p):
    with open(p, encoding="utf-8") as f:
        return [ln.strip() for ln in f if ln.strip()]

pos, neg = read_txt(POS_FILE), read_txt(NEG_FILE)
df = pd.DataFrame({"text": pos + neg, "label": [1]*len(pos) + [0]*len(neg)}).sample(frac=1, random_state=SEED)

train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=SEED)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df["label"], random_state=SEED)
print(f"Train={len(train_df)}, Val={len(val_df)}, Test={len(test_df)}")


Train=9445, Val=1181, Test=1181


# Train teacher xlmRand save best

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, f1_score
import torch.nn as nn

TEACHER_MODEL_ID = "xlm-roberta-base"
NUM_LABELS = 2
MAX_LEN = 128
BATCH_SIZE = 16
EPOCHS = 3
LR = 2e-5
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01

teacher_tok = AutoTokenizer.from_pretrained(TEACHER_MODEL_ID)
teacher = AutoModelForSequenceClassification.from_pretrained(TEACHER_MODEL_ID, num_labels=NUM_LABELS).to(DEVICE)

class TextDataset(Dataset):
    def __init__(self, df, tok, max_len):
        self.texts = df.text.tolist(); self.labels = df.label.tolist()
        self.tok, self.max_len = tok, max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, i):
        enc = self.tok(self.texts[i], truncation=True, padding="max_length", max_length=self.max_len, return_tensors="pt")
        return {"input_ids": enc["input_ids"].squeeze(0),
                "attention_mask": enc["attention_mask"].squeeze(0),
                "labels": torch.tensor(self.labels[i], dtype=torch.long)}

train_loader = DataLoader(TextDataset(train_df, teacher_tok, MAX_LEN), batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(TextDataset(val_df, teacher_tok, MAX_LEN), batch_size=BATCH_SIZE)
test_loader_ = DataLoader(TextDataset(test_df, teacher_tok, MAX_LEN), batch_size=BATCH_SIZE)

opt = AdamW(teacher.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
steps = len(train_loader)*EPOCHS
sch = get_linear_schedule_with_warmup(opt, int(WARMUP_RATIO*steps), steps)

best_f1 = -1
for ep in range(1, EPOCHS+1):
    teacher.train(); total_loss = 0
    for b in tqdm(train_loader, desc=f"Teacher Epoch {ep}/{EPOCHS}"):
        b = {k:v.to(DEVICE) for k,v in b.items()}
        out = teacher(**b)
        loss = out.loss
        loss.backward()
        opt.step(); sch.step(); opt.zero_grad()
        total_loss += loss.item()

    # Validation
    teacher.eval(); preds, gold = [], []
    with torch.no_grad():
        for b in val_loader:
            b = {k:v.to(DEVICE) for k,v in b.items()}
            out = teacher(**b)
            preds += out.logits.argmax(-1).cpu().tolist()
            gold  += b["labels"].cpu().tolist()
    acc = accuracy_score(gold, preds)
    f1m = f1_score(gold, preds, average="macro")
    print(f"Val Acc={acc:.4f} | F1_macro={f1m:.4f}")
    if f1m > best_f1:
        best_f1 = f1m
        teacher.save_pretrained(WORK_DIR/"fine_tuned_xlmr")
        teacher_tok.save_pretrained(WORK_DIR/"fine_tuned_xlmr")
        print("✅ Saved best teacher.")

# Evaluate on test
teacher.eval(); preds, gold = [], []
with torch.no_grad():
    for b in test_loader_:
        b = {k:v.to(DEVICE) for k,v in b.items()}
        out = teacher(**b)
        preds += out.logits.argmax(-1).cpu().tolist()
        gold  += b["labels"].cpu().tolist()
print("Teacher Test Acc=", accuracy_score(gold,preds), "MacroF1=", f1_score(gold,preds,average="macro"))


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

...

Val Acc=0.9526 | F1_macro=0.9416
✅ Saved best teacher.


Teacher Epoch 2/3:   0%|          | 0/591 [00:00<?, ?it/s]

Val Acc=0.9602 | F1_macro=0.9506
✅ Saved best teacher.


Teacher Epoch 3/3:   0%|          | 0/591 [00:00<?, ?it/s]

Val Acc=0.9636 | F1_macro=0.9548
✅ Saved best teacher.
Teacher Test Acc= 0.9576629974597799 MacroF1= 0.9478168809959774


# Transliteration KD Dataset

In [4]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

def transliterate_bn_text(text: str):
    try:
        return transliterate(text, sanscript.BENGALI, sanscript.ITRANS)
    except Exception:
        return text

STUDENT_MODEL_ID = "distilroberta-base"
from transformers import AutoTokenizer
student_tok = AutoTokenizer.from_pretrained(STUDENT_MODEL_ID)

class KDDataset(Dataset):
    def __init__(self, df, t_tok, s_tok, max_len):
        self.texts, self.labels = df.text.tolist(), df.label.tolist()
        self.ttok, self.stok, self.max_len = t_tok, s_tok, max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, i):
        txt_bn = self.texts[i]
        txt_en = transliterate_bn_text(txt_bn)
        t = self.ttok(txt_bn, truncation=True, padding="max_length", max_length=self.max_len, return_tensors="pt")
        s = self.stok(txt_en, truncation=True, padding="max_length", max_length=self.max_len, return_tensors="pt")
        return {
            "t_input_ids": t["input_ids"].squeeze(0),
            "t_attention_mask": t["attention_mask"].squeeze(0),
            "s_input_ids": s["input_ids"].squeeze(0),
            "s_attention_mask": s["attention_mask"].squeeze(0),
            "labels": torch.tensor(self.labels[i], dtype=torch.long)
        }

def pad_collate(batch, t_pad, s_pad):
    out = {}
    for k in batch[0]:
        if k == "labels": out[k] = torch.stack([b[k] for b in batch])
        elif k.startswith("t_"):
            padv = 0 if "attention" in k else t_pad
            out[k] = nn.utils.rnn.pad_sequence([b[k] for b in batch], batch_first=True, padding_value=padv)
        elif k.startswith("s_"):
            padv = 0 if "attention" in k else s_pad
            out[k] = nn.utils.rnn.pad_sequence([b[k] for b in batch], batch_first=True, padding_value=padv)
    return out

BATCH_SIZE = 16
train_loader = DataLoader(KDDataset(train_df, teacher_tok, student_tok, MAX_LEN), batch_size=BATCH_SIZE, shuffle=True,
                          collate_fn=lambda b: pad_collate(b, teacher_tok.pad_token_id, student_tok.pad_token_id))
val_loader   = DataLoader(KDDataset(val_df, teacher_tok, student_tok, MAX_LEN), batch_size=BATCH_SIZE,
                          collate_fn=lambda b: pad_collate(b, teacher_tok.pad_token_id, student_tok.pad_token_id))
test_loader  = DataLoader(KDDataset(test_df, teacher_tok, student_tok, MAX_LEN), batch_size=BATCH_SIZE,
                          collate_fn=lambda b: pad_collate(b, teacher_tok.pad_token_id, student_tok.pad_token_id))
print("✅ KD data ready.")


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

✅ KD data ready.


In [5]:
import torch.nn as nn
from transformers import AutoModel

class StudentClassifier(nn.Module):
    def __init__(self, base_model_id, num_labels=2):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(base_model_id)
        H = self.encoder.config.hidden_size
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(H, num_labels)
    def forward(self, input_ids, attention_mask):
        out = self.encoder(input_ids=input_ids, attention_mask=attention_mask,
                           output_hidden_states=True, output_attentions=True, return_dict=True)
        cls = out.last_hidden_state[:,0,:]
        logits = self.classifier(self.dropout(cls))
        return {"logits": logits, "hidden_states": out.hidden_states, "attentions": out.attentions}

student = StudentClassifier(STUDENT_MODEL_ID).to(DEVICE)
print("✅ Student ready.")


model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

✅ Student ready.


# Student model + Logit + Hidden + Attention KD loss

In [6]:
import torch.nn.functional as F

class KDLossFull(nn.Module):
    def __init__(self, T=3.0, alpha=0.5, gamma_h=1.0, gamma_a=1.0):
        super().__init__()
        self.T, self.alpha, self.gamma_h, self.gamma_a = T, alpha, gamma_h, gamma_a
        self.ce = nn.CrossEntropyLoss()
        self.kld = nn.KLDivLoss(reduction="batchmean")
        self.mse = nn.MSELoss()
    @staticmethod
    def map_layers(n_s, n_t):
        s_idx = list(range(1, n_s + 1))
        t_idx = torch.linspace(1, n_t, steps=n_s).round().long().tolist()
        return list(zip(s_idx, t_idx))
    def forward(self, s_pack, t_pack, labels):
        logits_s, logits_t = s_pack["logits"], t_pack["logits"]
        hard = self.ce(logits_s, labels)
        soft = self.kld(F.log_softmax(logits_s/self.T, dim=-1),
                        F.softmax(logits_t/self.T, dim=-1)) * (self.T**2)
        total = (1-self.alpha)*hard + self.alpha*soft

        hs, ht = s_pack.get("hidden_states", []), t_pack.get("hidden_states", [])
        if hs and ht:
            pairs = self.map_layers(len(hs)-1, len(ht)-1)
            h_losses = [self.mse(hs[i_s][:,:min(hs[i_s].size(1),ht[i_t].size(1)),:],
                                 ht[i_t][:,:min(hs[i_s].size(1),ht[i_t].size(1)),:])
                        for i_s,i_t in pairs]
            total += self.gamma_h * torch.stack(h_losses).mean()

        as_, at_ = s_pack.get("attentions", []), t_pack.get("attentions", [])
        if as_ and at_:
            pairs = self.map_layers(len(as_), len(at_))
            a_losses = []
            for i_s,i_t in pairs:
                try:
                    s_a, t_a = as_[i_s], at_[i_t]
                    if s_a.dim()!=4 or t_a.dim()!=4: continue
                    L=min(s_a.size(-1),t_a.size(-1))
                    a_losses.append(self.mse(s_a[:,:,:L,:L], t_a[:,:,:L,:L]))
                except Exception: continue
            if a_losses: total += self.gamma_a * torch.stack(a_losses).mean()
        return total

criterion = KDLossFull()
print("✅ KD loss ready.")


✅ KD loss ready.


# KD training loop enabling attentions

In [7]:
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

LR, EPOCHS, WARMUP, WD, PATIENCE = 3e-5, 5, 0.1, 0.01, 2

opt = AdamW(student.parameters(), lr=LR, weight_decay=WD)
steps = len(train_loader)*EPOCHS
sch = get_linear_schedule_with_warmup(opt, int(WARMUP*steps), steps)

teacher.eval(); [setattr(p, "requires_grad", False) for p in teacher.parameters()]

def metrics(p,g):
    return {"accuracy": accuracy_score(g,p),
            "f1_macro": f1_score(g,p,average="macro"),
            "f1_weighted": f1_score(g,p,average="weighted")}

@torch.no_grad()
def eval_student(loader):
    student.eval(); preds,gold=[],[]
    for b in loader:
        b={k:v.to(DEVICE) for k,v in b.items()}
        out=student(b["s_input_ids"],b["s_attention_mask"])
        preds+=out["logits"].argmax(-1).cpu().tolist(); gold+=b["labels"].cpu().tolist()
    return metrics(np.array(preds),np.array(gold))

best_f1, wait = -1, 0
for ep in range(1,EPOCHS+1):
    student.train(); run=0.0
    for b in tqdm(train_loader,desc=f"KD Epoch {ep}/{EPOCHS}"):
        labels=b["labels"].to(DEVICE)
        s_out=student(b["s_input_ids"].to(DEVICE),b["s_attention_mask"].to(DEVICE))
        with torch.no_grad():
            t_raw=teacher(b["t_input_ids"].to(DEVICE),b["t_attention_mask"].to(DEVICE),
                          output_hidden_states=True,output_attentions=True,return_dict=True)
            t_out={"logits":t_raw.logits,"hidden_states":t_raw.hidden_states,"attentions":t_raw.attentions}
        loss=criterion(s_out,t_out,labels)
        loss.backward(); nn.utils.clip_grad_norm_(student.parameters(),1.0)
        opt.step(); sch.step(); opt.zero_grad(); run+=loss.item()
    val=eval_student(val_loader)
    print(f"[KD] Loss={run/len(train_loader):.4f} | Val F1m={val['f1_macro']:.4f} | Acc={val['accuracy']:.4f}")
    if val["f1_macro"]>best_f1:
        best_f1=val["f1_macro"]; wait=0; torch.save(student.state_dict(),WORK_DIR/"student_best.pt")
    else:
        wait+=1
        if wait>=PATIENCE: print("⏸️ Early stopping."); break

student.load_state_dict(torch.load(WORK_DIR/"student_best.pt", map_location=DEVICE))
student.eval()
print("✅ KD training complete.")


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


KD Epoch 1/5:   0%|          | 0/591 [00:00<?, ?it/s]

RobertaSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.
XLMRobertaSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


[KD] Loss=1.8499 | Val F1m=0.8750 | Acc=0.8916


KD Epoch 2/5:   0%|          | 0/591 [00:00<?, ?it/s]

...

[KD] Loss=0.6049 | Val F1m=0.9193 | Acc=0.9348


KD Epoch 4/5:   0%|          | 0/591 [00:00<?, ?it/s]

[KD] Loss=0.5158 | Val F1m=0.9238 | Acc=0.9399


KD Epoch 5/5:   0%|          | 0/591 [00:00<?, ?it/s]

[KD] Loss=0.4512 | Val F1m=0.9237 | Acc=0.9382
✅ KD training complete.


In [8]:
from scipy.special import softmax
from scipy.spatial.distance import cosine

@torch.no_grad()
def eval_model(model, loader, mode):
    preds,gold=[],[]
    for b in loader:
        b={k:v.to(DEVICE) for k,v in b.items()}
        inp={"input_ids":b["t_input_ids"],"attention_mask":b["t_attention_mask"]} if mode=="teacher" else \
            {"input_ids":b["s_input_ids"],"attention_mask":b["s_attention_mask"]}
        out=model(**inp); logits=out["logits"] if isinstance(out,dict) else out.logits
        preds+=logits.argmax(-1).cpu().tolist(); gold+=b["labels"].cpu().tolist()
    return metrics(preds,gold)

teacher_test=eval_model(teacher,test_loader,"teacher")
student_test=eval_model(student,test_loader,"student")
print("[Teacher][Test]:",teacher_test)
print("[Student][Test]:",student_test)

@torch.no_grad()
def alignment_metrics(teacher,student,loader):
    cos, corr, agree=[],[],[]
    for b in loader:
        b={k:v.to(DEVICE) for k,v in b.items()}
        t=teacher(b["t_input_ids"],b["t_attention_mask"])
        s=student(b["s_input_ids"],b["s_attention_mask"])
        t_logits,s_logits=t.logits.cpu().numpy(),s["logits"].cpu().numpy()
        t_probs,s_probs=softmax(t_logits,axis=-1),softmax(s_logits,axis=-1)
        for tl,sl,tp,sp in zip(t_logits,s_logits,t_probs,s_probs):
            cos.append(1-cosine(tl,sl))
            corr.append(np.corrcoef(tp,sp)[0,1])
            agree.append(np.argmax(tp)==np.argmax(sp))
    return {"logit_cosine":float(np.nanmean(cos)),
            "prob_corr":float(np.nanmean(corr)),
            "pred_alignment":float(np.mean(agree))}

align=alignment_metrics(teacher,student,test_loader)
print(f"""
🧩 Alignment Results
  Logit cosine : {align['logit_cosine']:.4f}
  Prob corr    : {align['prob_corr']:.4f}
  Agreement    : {align['pred_alignment']:.4f}
""")

SAVE_DIR = WORK_DIR / "distilroberta_student_translit_kd"
SAVE_DIR.mkdir(exist_ok=True)
torch.save(student.state_dict(), SAVE_DIR/"pytorch_model.bin")
student_tok.save_pretrained(SAVE_DIR)
json.dump({"teacher_test":teacher_test,"student_test":student_test,"alignment":align},
          open(WORK_DIR/"metrics_distilroberta.json","w"),indent=2)
print("✅ Saved model + metrics.")


[Teacher][Test]: {'accuracy': 0.9576629974597799, 'f1_macro': 0.9478168809959774, 'f1_weighted': 0.9577781567166664}
[Student][Test]: {'accuracy': 0.9356477561388654, 'f1_macro': 0.9191876629644036, 'f1_weighted': 0.9352154084757462}

🧩 Alignment Results
  Logit cosine : 0.8833
  Prob corr    : 0.8950
  Agreement    : 0.9475

✅ Saved model + metrics.
